In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, UpSampling2D, BatchNormalization
from keras.metrics import RootMeanSquaredError
from keras.optimizers import AdamW
from keras import backend as K

In [2]:
X_train = np.load('../y_smp_train.npy')
y_train = np.load('../pars_smp_train.npy')
X_test = np.load('../y_smp_test.npy')

In [3]:
X_train.shape, y_train.shape

((1000000, 200, 3), (1000000, 15, 1))

In [4]:
def quantile_loss_5(y_true, y_pred):
    error = y_true - y_pred
    loss = K.mean(K.maximum(0.5 * error, (0.5 - 1) * error))
    return loss

In [5]:
time_steps = 200
num_features = 3

In [6]:
model = Sequential()

model.add(Conv2D(16, (3,3), activation='relu', input_shape=(time_steps, num_features, 1), padding='same'))
model.add(UpSampling2D((2,2)))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2,2)))
model.add(Conv2D(32, (12,6), activation='relu', strides=(6, 1)))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Conv2D(16, (3,3), activation='relu', strides=(2, 1)))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
model.add(Dense(15))

opt = AdamW(learning_rate=0.0001)
model.compile(loss=quantile_loss_5, optimizer=opt, metrics=[RootMeanSquaredError()])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 200, 3, 16)        160       
                                                                 
 up_sampling2d (UpSampling2  (None, 400, 6, 16)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 400, 6, 32)        4640      
                                                                 
 up_sampling2d_1 (UpSamplin  (None, 800, 12, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 132, 7, 32)        73760     
                                                                 
 max_pooling2d (MaxPooling2  (None, 66, 4, 32)         0

In [ ]:
model.fit(X_train, y_train, batch_size=128, epochs=25)

In [ ]:
pred = model.predict(X_test)

In [ ]:
np.save('test_pred_05', pred)

In [ ]:
model.save("model_05.h5")